In [1]:
import torch
import torch.nn as nn

checkpoint_path = 'malaria_AI.pth'
checkpoint = torch.load(checkpoint_path)

print("Checkpoint keys:", checkpoint.keys())

if isinstance(checkpoint, dict):
    for key in checkpoint.keys():
        print(key)
elif isinstance(checkpoint, torch.nn.Module):
    print(checkpoint)
else:
    print("Unexpected checkpoint format:", type(checkpoint))


Checkpoint keys: odict_keys(['conv_block_1.0.weight', 'conv_block_1.0.bias', 'conv_block_1.2.weight', 'conv_block_1.2.bias', 'conv_block_2.0.weight', 'conv_block_2.0.bias', 'conv_block_2.2.weight', 'conv_block_2.2.bias', 'classifier.1.weight', 'classifier.1.bias'])
conv_block_1.0.weight
conv_block_1.0.bias
conv_block_1.2.weight
conv_block_1.2.bias
conv_block_2.0.weight
conv_block_2.0.bias
conv_block_2.2.weight
conv_block_2.2.bias
classifier.1.weight
classifier.1.bias


C:\Users\olawa\AppData\Local\Temp\ipykernel_18424\594353613.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [2]:
"""
if a GPU, computation process will be allocated to the GPU and if not, it will be pushed to the CPU
"""
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
class CNN_1(nn.Module):
    
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 289, out_features=output_shape)
        )
    
    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        
# Instantiate the model
input_shape = 3 
hidden_units = 12  
output_shape = 2  
model = CNN_1(input_shape, hidden_units, output_shape).to(device)

# Load the state dictionary
model.load_state_dict(torch.load('malaria_AI.pth'))
model.eval()

C:\Users\olawa\AppData\Local\Temp\ipykernel_18424\3005526871.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('malaria_AI.pth'))


CNN_1(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3468, out_features=2, bias=True)
  )
)

In [4]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2

#Preprocessing function, resizes the images and convert them into tensors
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device)

In [5]:
# Function to select a random image from the specified folder
def get_random_image(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    random_subfolder = random.choice(subfolders)
    images = [os.path.join(random_subfolder, img) for img in os.listdir(random_subfolder) if img.endswith('.png')] #all the images are .png files
    return random.choice(images)

def predict(image_tensor, model):
    with torch.no_grad():
        output = model(image_tensor)
    return output

In [6]:
#if your local system doesnt have a gpu, the code below is still going to work. because they have been moved to the cup

In [7]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

def generate_and_show_gradcam(model, target_layer, image_tensor, predicted_label):
    # Instantiating the Grad-CAM object
    cam = GradCAM(model=model, target_layers=[target_layer])

    # Generate the Grad-CAM visualization
    grayscale_cam = cam(input_tensor=image_tensor)
    grayscale_cam = grayscale_cam[0, :]  

    original_image = image_tensor.squeeze().cpu().permute(1, 2, 0).numpy()
    original_image = original_image / original_image.max()
    visualization = show_cam_on_image(original_image, grayscale_cam, use_rgb=True, colormap= cv2.COLORMAP_JET, image_weight=0.7)
    #the image_weight can be experimented with, use values between 0.1 and 1

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(original_image)
    ax[0].axis('off')
    ax[0].set_title(f'{predicted_label} | Original Image')

    ax[1].imshow(visualization)
    ax[1].axis('off')
    ax[1].set_title(f'{predicted_label} | GradCAM')

    plt.show()

# Example usage
folder_path = 'Test'
random_image_path = get_random_image(folder_path)
image_tensor = preprocess_image(random_image_path).to(device)
output = predict(image_tensor, model)
_, predicted = torch.max(output, 1)
class_labels = {0: 'parasite', 1: 'normal'}
predicted_label = class_labels[predicted.item()]
print(f'Predicted class for {random_image_path}: {predicted_label}')

target_layer = model.conv_block_2[-1]
generate_and_show_gradcam(model, target_layer, image_tensor, predicted_label)
#to view succesive images, you dont have to run all the cells again, just the last cell, unlike the former code.

ModuleNotFoundError: No module named 'pytorch_grad_cam'